In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)

In [2]:
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False)
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        model.fit(X_tr, y_tr)
        
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)
    
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)
    
    return train_fold_pred, test_pred_mean

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


knn_clf = KNeighborsClassifier(n_neighbors=4)
ada_clf = AdaBoostClassifier(n_estimators=100)
rf_clf = RandomForestClassifier(n_estimators=100)
dt_clf = DecisionTreeClassifier()


knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)

In [4]:
X_train_final = np.concatenate((knn_train, ada_train, rf_train, dt_train), axis=1)
X_test_final = np.concatenate((knn_test, ada_test, rf_test, dt_test), axis=1)

print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')
print(f'X_train_final shape: {X_train_final.shape}, X_test_final shape: {X_test_final.shape}')

X_train shape: (455, 30), X_test shape: (114, 30)
X_train_final shape: (455, 4), X_test_final shape: (114, 4)


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_final = LogisticRegression(C=10)
lr_final.fit(X_train_final, y_train)
stack_final = lr_final.predict(X_test_final)

print(f'Final accuracy: {accuracy_score(y_test, stack_final):.4f}')

Final accuracy: 0.9737
